<a href="https://colab.research.google.com/github/Sankalp1233/Code/blob/main/BERT_sliding_window.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [ ]:
from google.colab import files
uploaded = files.upload() 
filename = "Spark_QA.txt"
new_file = uploaded[filename].decode("utf-8")
sentences = new_file.split('\n')
context = []
questions = []
count = 0
for i in sentences:
    if count % 2 == 1:
        context.append(i)
    else:
      questions.append(i)
    count += 1
import pandas as pd
questions = pd.DataFrame(questions)
questions.columns = ["questions"]
context = pd.DataFrame(context)
context.columns = ["context"]
horizontal_stack = pd.concat([questions, context], axis=1)
horizontal_stack.columns = ['question','context']
horizontal_stack = pd.DataFrame(horizontal_stack)
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(horizontal_stack,test_size = 0.05)
!pip3 install datasets
from datasets import load_dataset
import pandas as pd
new_train_df = pd.DataFrame(train_df)
# new_train_df.to_csv(r'C:\Desktop\Sonny_Simons Fou ndation\train.csv', index = False, header=True)
new_val_df = pd.DataFrame(val_df)
# new_val_df.to_csv(r'C:\Desktop\Sonny_Simons Fou ndation\test.csv', index = False, header=True)
vertical_stack = pd.concat([new_train_df, new_val_df], axis=0)
from google.colab import drive
drive.mount('drive')
new_train_df.to_csv('train.csv')
new_val_df.to_csv('test.csv')
from datasets import load_dataset
train_test_csv_dataset = load_dataset('csv', data_files={'train': 'train.csv',
                                              'test': 'test.csv'})

Saving Spark_QA.txt to Spark_QA (1).txt


Using custom data configuration default-22f222903204abc1


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-22f222903204abc1/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [ ]:
# !pip3 install string
def chatbot():
    model_checkpoint = "distilbert-base-uncased"
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    question = input("Ask a question: ")
    from string import ascii_letters
    question = ''.join([letter for letter in question if letter in ascii_letters])
    answer = []
    for i in range(len(train_test_csv_dataset["train"])):
        example = train_test_csv_dataset["train"][i]
        example['question'] = ''.join([letter for letter in example['question'] if letter in ascii_letters])
        # print(example['question'])
        if question.lower().replace(" ", "") == example['question'].lower().replace(" ", ""):
           tokenized_example = tokenizer(
                               question,
                               example["context"],
                               max_length=len(example["context"]) + len(example['question']),
                               truncation="only_second",
                               return_overflowing_tokens=True,
                               return_offsets_mapping=True,
                               stride = 128)
           answers = example["context"]
           start_char = 0
           end_char = start_char + len(answers)
           sequence_ids = tokenized_example.sequence_ids()
           # Start token index of the current span in the text.
           token_start_index = 0
           while sequence_ids[token_start_index] != 1:
                 token_start_index += 1
           # End token index of the current span in the text.
           token_end_index = len(tokenized_example["input_ids"][0]) - 1
           while sequence_ids[token_end_index] != 1:
                 token_end_index -= 1
          # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
           offsets = tokenized_example["offset_mapping"][0]
           if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
             # Move the token_start_index and token_end_index to the two ends of the answer.
             # Note: we could go after the last offset if the answer is the last word (edge case).
              while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
              start_position = token_start_index - 1
              while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
              end_position = token_end_index + 1
              answer.append(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
    for i in range(len(train_test_csv_dataset["test"])):
        example = train_test_csv_dataset["test"][i]
        example['question']= ''.join([letter for letter in example['question'] if letter in ascii_letters])
        # print(example['question'])
        if question.lower().replace(" ", "") == example['question'].lower().replace(" ", ""):
           tokenized_example = tokenizer(
                                question,
                                example["context"],
                                max_length= len(example["context"]) + len(example['question']),
                                truncation="only_second",
                                return_overflowing_tokens=True,
                                return_offsets_mapping=True,
                                stride = 128)
           answers = example["context"]
           start_char = 0
           end_char = start_char + len(answers)
           sequence_ids = tokenized_example.sequence_ids()
           # Start token index of the current span in the text.
           token_start_index = 0
           while sequence_ids[token_start_index] != 1:
                 token_start_index += 1
           # End token index of the current span in the text.
           token_end_index = len(tokenized_example["input_ids"][0]) - 1
           while sequence_ids[token_end_index] != 1:
                 token_end_index -= 1
           # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
           offsets = tokenized_example["offset_mapping"][0]
           if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
               # Move the token_start_index and token_end_index to the two ends of the answer.
               # Note: we could go after the last offset if the answer is the last word (edge case).
              while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
              start_position = token_start_index - 1
              while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
              end_position = token_end_index + 1
              answer.append(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
    TDNA_string = "What can my TDNA do?"
    DNA_string = "What can we do from DNA with thousands of families across the country?"
    gene_string = "What makes it easier to spot differences in genes?"
    weather_string = "What is the weather today?"
    last_name_string = "What is my last name?"
    first_name_string = "What is my first name?"
    children_and_dep_adults_string = "To participate in SPARK, what do children ages 10 through 17 and dependent adults need to do?"
    saliva_collection_string = "What is saliva collection used for?"
    military_string = "If I am an individual who is in the military overseas what address should I use for registration?"
    address_string = "How do I make changes to my address?"
    join_SPARK_string = "What happens when you join SPARK?"
    NDAR_full_string = "What is National Database for Autism Research?"
    NDAR_string = "What is NDAR?"
    NDAR_full_string2 = "What is the National Database for Autism Research?"
    NDAR_string2 = "What is the NDAR?"
    Research_data_string = "What happens before any of my research data is shared for other autism research?"
    Sequencing_data_string = "Where is sequencing data stored?"
    DNA_sequencing_string = "What portion of DNA does SPARK look at for DNA sequencing?"
    Outside_researcher_string = "What must an outside researcher do before they are allowed to access SPARK's DNA database?"
    contact_info_string = "Will Spark share my contact information?"
    TDNA_string = ''.join([letter for letter in TDNA_string if letter in ascii_letters])
    DNA_string = ''.join([letter for letter in DNA_string if letter in ascii_letters])
    gene_string = ''.join([letter for letter in gene_string if letter in ascii_letters])
    weather_string = ''.join([letter for letter in weather_string if letter in ascii_letters])
    last_name_string = ''.join([letter for letter in last_name_string if letter in ascii_letters])
    first_name_string = ''.join([letter for letter in first_name_string if letter in ascii_letters])
    children_and_dep_adults_string = ''.join([letter for letter in children_and_dep_adults_string if letter in ascii_letters])
    saliva_collection_string = ''.join([letter for letter in saliva_collection_string if letter in ascii_letters])
    military_string = ''.join([letter for letter in military_string if letter in ascii_letters])
    address_string = ''.join([letter for letter in address_string if letter in ascii_letters])
    join_SPARK_string = ''.join([letter for letter in join_SPARK_string if letter in ascii_letters])
    NDAR_full_string = ''.join([letter for letter in NDAR_full_string if letter in ascii_letters])
    NDAR_string = ''.join([letter for letter in NDAR_string if letter in ascii_letters])
    NDAR_full_string2 = ''.join([letter for letter in NDAR_full_string2 if letter in ascii_letters])
    NDAR_string2 = ''.join([letter for letter in NDAR_string2 if letter in ascii_letters])
    Research_data_string = ''.join([letter for letter in Research_data_string if letter in ascii_letters])
    Sequencing_data_string = ''.join([letter for letter in Sequencing_data_string if letter in ascii_letters])
    DNA_sequencing_string = ''.join([letter for letter in DNA_sequencing_string if letter in ascii_letters])
    Outside_researcher_string = ''.join([letter for letter in Outside_researcher_string if letter in ascii_letters])
    contact_info_string = ''.join([letter for letter in contact_info_string if letter in ascii_letters])
    if   len(answer)!=0:
         print(' '.join([str(elem) for elem in answer]))
    elif question.lower().replace(" ", "") == TDNA_string.lower().replace(" ", ""):
         print('spark the next genetic discovery')
    elif question.lower().replace(" ", "") == DNA_string.lower().replace(" ", ""):
         print('learn more about genes related to autism and find new ones')
    elif question.lower().replace(" ", "") == gene_string.lower().replace(" ", ""):
         print('DNA from thousands of families across the country')
    elif question.lower().replace(" ", "") == weather_string.lower().replace(" ", ""):
         print('This is not related to spark please ask another question')
    elif question.lower().replace(" ", "") == last_name_string.lower().replace(" ", ""):
         print("This is not related to spark please ask another question")
    elif question.lower().replace(" ", "") == first_name_string.lower().replace(" ", ""):
         print("This is not related to spark please ask another question")
    elif question.lower().replace(" ","") == children_and_dep_adults_string.lower().replace(" ",""):
         print("asked to assent, if they are capable. Assent is a child's or dependent's agreement that he or she is willing to participate in research.")
    elif question.lower().replace(" ","") == saliva_collection_string.lower().replace(" ",""):
         print("It is a common method for research ranging from large scientific studies to individual personalized medicine tests.")
    elif question.lower().replace(" ","") == military_string.lower().replace(" ",""):
         print("Use your current FPO or APO address.")
    elif question.lower().replace(" ","") == address_string.lower().replace(" ",""):
         print("You can do so by visiting your SPARK dashboard.")
    elif question.lower().replace(" ","") == join_SPARK_string.lower().replace(" ",""):
         print("When you join SPARK, you allow the Simons Foundation Autism Research Initiative (SFARI) to share your research data with the National Database for Autism Research (NDAR).")
    elif question.lower().replace(" ","") == NDAR_full_string.lower().replace(" ","") or question.lower().replace(" ","") == NDAR_string.lower().replace(" ","") or question.lower().replace(" ","") == NDAR_string2.lower().replace(" ","") or question.lower().replace(" ","") == NDAR_full_string2.lower().replace(" ",""):
         print("NDAR is a central research data source for autism researchers from around the country. Data in NDAR is stored without any information that could identify you.")
    elif question.lower().replace(" ","") == Research_data_string.lower().replace(" ",""):
         print("all information that could identify you is replaced with a global unique identifier (GUID). GUIDs allow researchers to share research data about you without exposing anything that could identify you.")
    elif question.lower().replace(" ","") == Sequencing_data_string.lower().replace(" ",""):
         print("All sequencing data is stored securely in the SPARK database.")
    elif question.lower().replace(" ","") == DNA_sequencing_string.lower().replace(" ",""):
         print("SPARK only looks at the portion of DNA that codes for proteins, because most genetic differences related to autism are found in this part of an individual’s DNA.")
    elif question.lower().replace(" ","") == Outside_researcher_string.lower().replace(" ",""):
         print("Outside researchers must go through an approval process before they are allowed to access SPARK’s DNA database.")
    elif question.lower().replace(" ","") == contact_info_string.lower().replace(" ",""):
         print("Your contact information will not be shared with researchers through SPARK’s research matching program unless you give us permission.")
    else:
      print("Please email info@SPARKforAutism.org")
chatbot()

Ask a question: Will you share my data?
Please email info@SPARKforAutism.org


In [ ]:
"""
Simple QA dialogue manager
"""
!pip3 install git+https://github.com/spokestack/wikiqa-python
# cd wikiqa-python
import tensorflow as tf
!pip3 install mediawiki
from mediawiki import MediaWiki
from spokestack.nlu.result import Result
from spokestack.nlu.tflite import TFLiteNLU
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering


class DialogueManager:
    """ Simple Question Answering Dialogue Manager """

    def __init__(self, log_path: str, base_model: str) -> None:
        self._wiki = MediaWiki()
        self._nlu = TFLiteNLU(log_path)
        self._tokenizer = AutoTokenizer.from_pretrained(base_model)
        self._answerer = TFAutoModelForQuestionAnswering.from_pretrained(base_model)

    def __call__(self, utterance: str) -> str:
        result = self._nlu(utterance)
        if result.intent == "ask.question":
            return self._answer(result)
        elif result.intent == "greet":
            return self.greet()
        elif result.intent == "command.exit":
            return self.exit()
        elif result.intent == "request.help":
            return self.help()
        else:
            return self.fallback()

    def _answer(self, result: Result) -> str:
        if result.slots:
            # get the tagged entity for page search
            entity = result.slots.get("entity").get("raw_value")
            # perform the search to find the wikipedia page
            entity = self._wiki.search(entity)[0]
            # get the page content to feed as context to the qa model
            passage = self._wiki.page(entity, auto_suggest=False).content
            # prepare qa model inputs
            inputs = self._tokenizer(
                result.utterance,
                passage,
                return_tensors="tf",
                padding=True,
                truncation=True,
            )
            # compute answer span
            start_scores, end_scores = self._answerer(inputs)
            start, end = tf.argmax(start_scores, -1)[0], tf.argmax(end_scores, -1)[0]
            # prepare the passage ids for slicing
            tokens = self._tokenizer.convert_ids_to_tokens(
                (inputs["input_ids"].numpy()[0])
            )
            # retrieve only the answer from the passage
            answer = self._tokenizer.convert_tokens_to_string(tokens[start : end + 1])
            return answer
        return "I don't have an answer for that"

    @staticmethod
    def greet() -> str:
        return "Hello, Ask me anything"

    @staticmethod
    def exit() -> str:
        return "Goodbye"

    @staticmethod
    def fallback() -> str:
        return (
            "I'm having trouble understanding your request, could you please "
            "repeat it"
        )

    @staticmethod
    def help() -> str:
        return "Ask a question like, how long is the amazon river?
manager = TextToSpeechManager(TextToSpeechClient(KEY_ID, KEY_SECRET), PyAudioOutput())
manager.synthesize("hello, world", "text", "demo-male")
@pipeline.event
def on_recognize(context):
    pipeline.pause()
    answer = dialogue_manager(context.transcript)
    manager.synthesize(answer, "text", "demo-male")
    pipeline.resume()


manager.synthesize(dialogue_manager.greet(), "text", "demo-male")
pipeline.start()
pipeline.run()

SyntaxError: ignored

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
for i, example in enumerate(train_test_csv_dataset["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = train_test_csv_dataset["train"][i]
print(i)
print(train_test_csv_dataset["train"][i]['question'])
tokenized_example = tokenizer(
    "What if I or my child can’t spit?",
    # example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride = doc_stride
)
answers = example["context"]
start_char = 0
end_char = start_char + len(answers)
sequence_ids = tokenized_example.sequence_ids()
# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")
print(example['question'])
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))

72
What if I or my child can’t spit?
13 90
What if I or my child can’t spit?
if you or your child are unable to spit, you can use the sponges that are included with each saliva kit. caregivers can collect the sample by sponging the inside of the person ’ s cheek and pressing the sponge against the notch of the funnel so that the saliva will flow into the tube. you can find video resources that will further explain the saliva collection process here.


In [ ]:
from google.colab import files
uploaded = files.upload() 
filename = "context.txt"
new_file = uploaded[filename].decode("utf-8")

Saving context.txt to context (1).txt


In [ ]:
!pip3 install transformers
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_example = tokenizer(
    "What is the purpose of Spark?",
    new_file,
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)

In [ ]:
for x in tokenized_example["input_ids"]:
    print(tokenizer.decode(x))

[CLS] what is the purpose of spark? [SEP] the purpose of spark is to help scientists find and better understand the potential causes of autism. as part of this effort, we study dna from people with autism and from their family members who may or may not have autism. we also study information about their health and things that can impact health like behavior and lifestyle. to succeed, spark needs many thousands of people with autism and their families to join. what we collect and learn will be shared with many autism researchers to help speed up the progress of autism research. you should take part in spark to help shape the future of autism research. your tdna could spark the next genetic discovery. with dna from thousands of families across the country, we will be able to learn more about genes that may be related to autism and discover new ones. in return, you will be able to get updates on the latest research, join other autism research studies, find possible genetic causes of autis

In [ ]:
sequence_ids = tokenized_example.sequence_ids()
answers = new_file
start_char = 0
end_char = start_char + len(answers)
print(end_char)
# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        print(token_start_index)
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
   print("The answer is not in this feature.")

31987
The answer is not in this feature.


In [ ]:
model_checkpoint = "distilbert-base-uncased"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
input_ids = tokenizer.encode("What is the purpose of Spark?", new_file)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [ ]:
# !pip3 uninstall torch
!pip3 install torch
import torch
# Run our example through the model.
start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 torch.tensor([segment_ids]),
                                 return_dict = False) # The segment IDs to differentiate question from answer_text


IndexError: ignored